In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [ ]:
# 输入数据
mnist=read_data_sets('/data',one_hot=True)

In [ ]:
# 定义网络的超参数
learning_rate=1e-3 #学习率
training_iters=20000 #训练的数据量
# 在训练和测试的时候，我们想要用 不同的数值，所以batch_size 定义为占位符
batch_size=tf.placeholder(tf.int32,[])
#batch_size=128 #每次训练多少数据
display_step=10  #每多少次显示一下当前状态


#每个时刻输入的特征为28维度，就是每个时刻输入一行，一行有28个元素
input_size=28
# 时序持续输入长度为28，即每次做一个预测，需要输入28行
time_step_size=28
# 每个隐含层的节点数
hidden_size=256
# LSTM_Layer的层数
layer_num=2
# 最后输出的分类类别数
class_num=10
# 输入的维度
n_input=784
#dropout
dropout=0.75
#设定数据占位符
x=tf.placeholder(tf.float32,[None,n_input])
y=tf.placeholder(tf.float32,[None,class_num])
keep_prob=tf.placeholder(tf.float32,[])

In [ ]:
#搭建LSTM模型
# 1.RNN的输入执行reshape。shape=(batch_size,time_step_size,input_size)
X=tf.reshape(x,[-1,time_step_size,input_size])
# 定义LSEM
def lstm_cell():
    # 2. 定义一层的LSTM_Cell，只需要设定hidden_size，他会自动匹配输入的X的维度
    cell=rnn.BasicLSTMCell(num_units=hidden_size,forget_bias=1.0,state_is_tuple=True)
    # 3.增加drop layer，一般只设置output_keep_prob
    return rnn.DropoutWrapper(cell=cell,input_keep_prob=1.0,output_keep_prob=keep_prob)
# 4.调用MultiRNNCell来实现多层LSTM
mlstm_cell=rnn.MultiRNNCell([lstm_cell() for _ in range(layer_num)],state_is_tuple=True)
# 5.用零来初始化state
init_state=mlstm_cell.zero_state(batch_size,dtype=tf.float32)
# 6.(1)方法一，调用dynamic_rnn来运行构建的网络
# time_major为False，表示output_shape=[batch_size,time_step_size,hidden_size]
# state_shape=[layer_num,2,batch_size,hidden_size]

# outputs,state=tf.nn.dynamic_rnn(mlstm_cell,inputs=X,initial_state=init_state,time_major=False)
# h_state=outputs[:,-1,:] 
# 或者 
#h_stat=state[-1][1]
#（2）方法二，按照时间展开计算
outputs = list()
state = init_state
with tf.variable_scope('RNN'):
    for timestep in range(time_step_size):
        if timestep > 0:
            tf.get_variable_scope().reuse_variables()
        # 这里的state保存了每一层 LSTM 的状态
        (cell_output, state) = mlstm_cell(X[:, timestep, :],state)
        outputs.append(cell_output)
h_state = outputs[-1]

In [ ]:
# 7.设置loss function和优化器
# softmax的连接权重矩阵和偏置
W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
bias = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
y_pre = tf.nn.softmax(tf.matmul(h_state, W) + bias)

# 损失和评估
cross=-tf.reduce_mean(y*tf.log(y_pre))
train_op=tf.train.AdamOptimizer(learning_rate).minimize(cross)

correct_pred=tf.equal(tf.argmax(y_pre,1),tf.argmax(y,1))
acc=tf.reduce_mean(tf.cast(correct_pred,'float'))

In [ ]:
# 训练
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(training_iters):
    _batch_size = 5
    batch=mnist.train.next_batch(_batch_size)
    if (i+1)*display_step==0:
        train_accuracy = sess.run(acc, feed_dict={
            x:batch[0], y: batch[1], keep_prob: 1.0, batch_size: _batch_size})
        print('Acc:%f'%(train_accuracy))
    sess.run(train_op, feed_dict={x: batch[0], y: batch[1], keep_prob: dropout, batch_size: _batch_size})

In [ ]:
# 可视化看看lstm怎么做分类
import matplotlib.pyplot as plt
X4=mnist.train.images[4]
print(mnist.train.labels[4])
img4=X4.reshape([28,28])
plt.imshow(img4,cmap='gray')
plt.show()

# 查看分类执行的过程
X4.shape=[-1,784]
y_batch=mnist.train.labels[4]
y_batch.shape=[-1,class_num]
X4_oupputs=np.array(session.run(outputs,feed_dict={X:X4,y:y_batch,keep_prob:1.0}))
print(X4_oupputs.shape)
X4_outputs.shape=[28,hidden_size]
h_W=sess.run(W,feed_dict={X:X4,y:y_batch,keep_prob:1.0})
h_bias=sess.run(W,feed_dict={X:X4,y:y_batch,keep_prob:1.0})
h_bias.shape=[-1,10]

# 显示output
bar_index=range(class_num)
for i in range(X4_outputs.shpae[0]):
    plt.subplot(7,4,i+1)
    X3_h_state=X4_outputs[i,:].reshape([-1,hidden_size])
    pro=sess.run(tf.nn.softmax(X3_h_state,h_W),h_bias)
    plt.bar(bar_index,pro[0],width=0.2,align='center')
    plt.axis('off')
plt.show()